**Importing necessary modules**

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery


**Installing What-If tool**

In [ ]:
!pip install witwidget

In [ ]:
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
project_id = 'unicef-aldeav'

In [ ]:
client = bigquery.Client(project=project_id)

**Retrieving natality dataset using BigQuery**

In [ ]:
sql = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""

df = client.query(sql).to_dataframe()

In [ ]:
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.686620,True,18,1,43.0
1,9.360828,True,32,1,41.0
2,8.437091,False,30,1,39.0
3,6.124442,False,24,1,40.0
4,7.125340,False,26,1,41.0


In [ ]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9989.000000,10000.000000,10000.000000,9890.000000
mean,7.297602,27.298900,1.034400,38.699798
std,1.291685,6.165838,0.192926,2.539957
min,0.612885,12.000000,1.000000,17.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.257702,50.000000,3.000000,47.000000


In [ ]:
df['is_male'].value_counts()

True     5150
False    4850
Name: is_male, dtype: int64

**Preparing the data**

In [ ]:
df = df.dropna()
df = shuffle(df, random_state = 2)

In [ ]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [ ]:
x, y = data, labels
x_train, x_test, y_train, y_test = train_test_split(x, y)

**Building and training model**

In [ ]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.columns),)),
    Dense(32, activation='relu'),
    Dense(1)
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 2,433
Trainable params: 2,433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
209/209 [==============================] - 1s 2ms/step - loss: 2.7053 - mae: 1.0925 - mse: 2.7053 - val_loss: 1.1660 - val_mae: 0.8334 - val_mse: 1.1660
Epoch 2/10
209/209 [==============================] - 0s 1ms/step - loss: 1.2824 - mae: 0.8832 - mse: 1.2824 - val_loss: 1.2435 - val_mae: 0.8698 - val_mse: 1.2435
Epoch 3/10
209/209 [==============================] - 0s 2ms/step - loss: 1.2724 - mae: 0.8765 - mse: 1.2724 - val_loss: 1.7464 - val_mae: 1.0537 - val_mse: 1.7464
Epoch 4/10
209/209 [==============================] - 0s 1ms/step - loss: 1.2476 - mae: 0.8730 - mse: 1.2476 - val_loss: 1.5607 - val_mae: 1.0009 - val_mse: 1.5607
Epoch 5/10
209/209 [==============================] - 0s 1ms/step - loss: 1.2311 - mae: 0.8652 - mse: 1.2311 - val_loss: 1.2793 - val_mae: 0.8946 - val_mse: 1.2793
Epoch 6/10
209/209 [==============================] - 0s 1ms/step - loss: 1.2152 - mae: 0.8582 - mse: 1.2152 - val_loss: 1.1161 - val_mae: 0.8275 - val_mse: 1.1161
Epoch 7/10
209/2

In [ ]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])
predictions.shape

(10, 1)

In [ ]:
for i in range(num_examples):
  print('Predicted val: ', predictions[i][0])
  print('Actual val: ', y_test.iloc[i])
  print()

Predicted val:  7.1558824
Actual val:  7.8705027534

Predicted val:  5.9338865
Actual val:  8.000575487979999

Predicted val:  5.221514
Actual val:  3.0622208191799998

Predicted val:  6.493481
Actual val:  4.62529825676

Predicted val:  6.7575135
Actual val:  7.936641432

Predicted val:  6.6600757
Actual val:  6.8122838958

Predicted val:  7.0130887
Actual val:  8.498820200099999

Predicted val:  7.331093
Actual val:  8.498820200099999

Predicted val:  7.058575
Actual val:  10.2074027306

Predicted val:  7.224786
Actual val:  8.24969784404



**Using What-If tool for visualisation**

In [ ]:
wit_data = pd.concat([x_test, y_test], axis=1)

In [ ]:
def custom_predict(examples_to_infer):
  preds = model.predict(examples_to_infer)
  return preds

In [ ]:
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  .set_custom_predict_fn(custom_predict)
  .set_target_feature('weight_pounds')
  .set_model_type('regression'))
WitWidget(config_builder, height=800)